# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']


M. Benisty  ->  M. Benisty  |  ['M. Benisty']
F. Zagaria  ->  F. Zagaria  |  ['F. Zagaria']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
N. Kurtovic  ->  N. Kurtovic  |  ['N. Kurtovic']
T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 98 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.14403


extracting tarball to tmp_2601.14403...

 done.


J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']


...

 done.


Issues with the citations
repeated bibliography entry: Law_ea_2023
Retrieving document from  https://arxiv.org/e-print/2601.15285


extracting tarball to tmp_2601.15285... done.


T. Henning  ->  T. Henning  |  ['T. Henning']


Error retrieving bib data for Hipparcos1997: 'author'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.14403-b31b1b.svg)](https://arxiv.org/abs/2601.14403) | **Ultraviolet spectroscopy reveals a hot and luminous companion to the Be star + black hole candidate MWC 656**  |
|| <mark>J. Müller-Horn</mark>, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2026-01-22*|
|*Comments*| *21 pages, 14 figures, submitted to A&A*|
|**Abstract**|            The Galactic Be star binary MWC 656 was long considered the only known Be star + black hole (BH) system, making it a critical benchmark for models of massive binary evolution and for the expected X-ray emission of Be+BH binaries. However, recent dynamical measurements cast doubt on the presence of a BH companion. We present new multi-epoch ultraviolet spectroscopy from the Hubble Space Telescope (HST), combined with high-resolution optical spectra, to reassess the nature of the companion. The far-ultraviolet spectra reveal high-ionisation features, including prominent N V and He II lines, which are absent in the spectra of normal Be stars and are indicative of a hot, luminous companion. Spectral modelling shows that these features cannot originate from the Be star or from an accretion disc around a compact object. Instead, we find that the data are best explained by a hot ($T_\mathrm{eff} \approx 85$ kK), compact, hydrogen-deficient star with strong wind signatures, consistent with an intermediate-mass stripped star. Our revised orbital solution and composite spectroscopic modelling yield a companion mass of $M_2 = 1.54^{+0.57}_{-0.46}\,\mathrm{M}_\odot$, definitively ruling out a BH and disfavouring a white dwarf. MWC 656 thus joins the growing class of Be + stripped star binaries. The system's unusual properties - including a high companion temperature and wind strength - extend the known parameter space of such binaries. The continued absence of confirmed OBe+BH binaries in the Galaxy highlights a growing tension with population synthesis models.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.15262-b31b1b.svg)](https://arxiv.org/abs/2601.15262) | **The Circumbinary Disk of HD 34700A: I. CO gas kinematics indicate spirals, infall, and vortex motions**  |
|| J. Stadler, et al. -- incl., <mark>M. Benisty</mark>, <mark>F. Zagaria</mark>, <mark>D. Fasano</mark>, <mark>N. Kurtovic</mark> |
|*Appeared on*| *2026-01-22*|
|*Comments*| *22 pages, 20 figures, accepted for publication in Astronomy and Astrophysics on January 16, 2026*|
|**Abstract**|            We present the first high-resolution ($\sim$ 0.14") Atacama Large Millimeter/submillimeter Array (ALMA) Band 6 dust continuum and CO molecular line emission observations of the quadruple system HD 34700. In particular, HD 34700AaAb is a spectroscopic binary ($M_{\rm{bin}}=4\,M_\odot$) surrounded by two low-mass companions at large separations. Its circumbinary disk is highly substructured, featuring numerous spiral arms and a large cavity observed in infrared (IR) scattered light. We analyzed the CO line channel and intensity moment maps. By fitting a Keplerian model to the line channel emission, we identified the residual motions and conducted a line spectra analysis. We resolved an asymmetric continuum crescent on top of a dust ring at 0.39" (138 au), colocated with the IR ring. The CO molecule line emissions trace a smaller cavity in gas, whose edge aligns with the inner rim of the ring detected in H$\alpha$ emission at 0.20" (65 au). The $^{12}$CO line emission and kinematics trace highly non-Keplerian motions ($\sim0.1\Delta\upsilon_{\rm kep}$), and these CO spiral features align well with the spiral structures in scattered light. The $^{12}$CO line spectra analysis reveals a streamer above the southeastern disk plane, likely falling onto the disk. The $^{13}$CO and C$^{18}$O kinematics largely follow the disk's underlying Keplerian rotation, while $^{13}$CO exhibits tentative signs of anticyclonic vortex flows at the continuum crescent location. Our multimolecular line study suggests that the circumbinary disk of HD 34700A is highly perturbed in its upper layers, possibly warped and influenced by infalling material. While late-stage infall may account for the IR spirals and the formation of the vortex through Rossby wave instability, an embedded massive companion within the cavity may also contribute to these features.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.15285-b31b1b.svg)](https://arxiv.org/abs/2601.15285) | **A radially broad collisional cascade in the debris disk of $γ$ Ophiuchi observed by JWST**  |
|| Y. Han, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2026-01-22*|
|*Comments*| *20 pages, 9 figures, 2 tables. Submitted to ApJ*|
|**Abstract**|            The A1V star $\gamma$ Oph, at a distance of 29.7 pc, is known from Spitzer imaging to host a debris disk with a large radial extent and from its spectral energy distribution to host inner warm dust. We imaged $\gamma$ Oph with JWST/MIRI at 15 and 25.5 microns, which reveal smooth and radially broad emission that extends to a radius of at least 250 au at 25.5 microns. In contrast to JWST findings of an inner small-grain component with distinct ringed substructures in Fomalhaut and Vega, the mid-infrared radial profile combined with prior ALMA imaging suggests a radially broad steady-state collisional cascade with the same grain size distribution throughout the disk. This further suggests that the system is populated by a radially broad planetesimal belt from tens of au or less to well over 200 au, rather than a narrow planetesimal belt from which the observed dust is displaced to appear broad. The disk is also found to be asymmetric, which could be modelled by a stellocentric offset corresponding to a small eccentricity of $\sim$0.02. Such a disk eccentricity could be induced by a mildly eccentric $<$$10\,M_\mathrm{Jup}$ giant planet outside 10 au, or a more eccentric companion up to stellar mass at a few au, without producing a resolvable radial gap in the disk.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.14661-b31b1b.svg)](https://arxiv.org/abs/2601.14661) | **Constraints on Axion-Like Particles from Ultra-High-Energy Observations of 3HWC J1908+063 with HAWC**  |
|| R. Alfaro, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-01-22*|
|*Comments*| **|
|**Abstract**|            Axion-like particles (ALPs) are hypothetical particles and compelling candidates for cold dark matter. Their existence could be probed through their conversions into photons in the presence of magnetic fields. In this work, we explore the effect of these photon-ALP conversions by searching for an attenuation in the observed gamma ray spectra of galactic sources that emit at energies of hundreds of TeV. We analyze data from the High-Altitude Water Cherenkov (HAWC) Observatory for the source 3HWC J1908+063. No evidence of photon-ALP conversions was found, and we set constraints on the ALP parameter space. Specifically, we derive exclusion limits for ALPs with masses in the range $10^{-8}~\mathrm{eV} \leq m_a \leq 10^{-6}~\mathrm{eV}$ and photon-ALP couplings in the range $10^{-12}~\mathrm{GeV}^{-1} \leq g_{a\gamma} \leq 10^{-10}~\mathrm{GeV}^{-1}$, based on HAWC observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.15012-b31b1b.svg)](https://arxiv.org/abs/2601.15012) | **A study of the high-inclination population in the Kuiper belt - V. Mean motion resonances beyond 50 AU**  |
|| <mark>J. Li</mark> |
|*Appeared on*| *2026-01-22*|
|*Comments*| *28 pages, 13 figures, 5 Tables, accepted for publication in MNRAS*|
|**Abstract**|            In this paper, we present the most comprehensive study to date on Neptune's mean-motion resonances (MMRs) in the distant Kuiper belt from 50 to 100 AU. Over 200 resonant Kuiper belt objects (KBOs) have been identified in this region, spanning resonances from the 2nd-order 1:3 MMR to the 22nd-order 7:29 MMR, with inclinations $i<40^\circ$. Building on these diverse distributions, we first analyse the dynamical features of numerous $m$:$n$ MMRs, providing an informative database that includes the possible eccentricity ($e$) range, resonance widths, resonance centres, and permissible $(e,i)$ distributions. We then conduct numerical simulations to explore the long-term stability of these MMRs. Our results show that: (1) resonators can occupy all 1:$n$ to 7:$n$ MMRs, with nearly any $n$ corresponding to the 50-100 AU region, including the farthest-out MMRs of 5:29 (24th-order), 6:35 (29th-order), and 7:40 (33rd-order). This suggests that KBOs could potentially exist in even higher-order MMRs than those currently observed. (2) For each set of $m$:$n$ resonances with the same $m$, resonators consistently exhibit inclinations up to $40^\circ$, while eccentricities remain strictly restricted below 0.7. (3) For the 1:3 and 1:4 MMRs, the leading population is less stable than the trailing population. Most interestingly, we discover a novel phenomenon of number reversal, where the higher-order, weaker 3:8 MMR (at semimajor axis $a\approx57.9$ AU) hosts more resonators, rather than fewer as expected, compared to the lower-order, stronger 3:7 MMR (at $a\approx53.0$ AU). Future observations, whether confirming or challenging this phenomenon, will offer valuable insight into the eccentricity and inclination distributions of primordial KBOs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.14403/./figures/HST_STIS_spec_zoom.png', 'tmp_2601.14403/./figures/HST_STIS_comparison_Be_subdwarfs.png', 'tmp_2601.14403/./figures/HST_STIS_comparison_CVs.png', 'tmp_2601.14403/./figures/orbit_fit_phasefolded_combined.png']
copying  tmp_2601.14403/./figures/HST_STIS_spec_zoom.png to _build/html/
copying  tmp_2601.14403/./figures/HST_STIS_comparison_Be_subdwarfs.png to _build/html/
copying  tmp_2601.14403/./figures/HST_STIS_comparison_CVs.png to _build/html/
copying  tmp_2601.14403/./figures/orbit_fit_phasefolded_combined.png to _build/html/
exported in  _build/html/2601.14403.md
    + _build/html/tmp_2601.14403/./figures/HST_STIS_spec_zoom.png
    + _build/html/tmp_2601.14403/./figures/HST_STIS_comparison_Be_subdwarfs.png
    + _build/html/tmp_2601.14403/./figures/HST_STIS_comparison_CVs.png
    + _build/html/tmp_2601.14403/./figures/orbit_fit_phasefolded_combined.png
found figures ['tmp_2601.15262/./figures/HD34700A_overview_allmom_csub_Tbr.png', 'tmp_2601.15

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.6}$
$\newcommand{\arraystretch}{1.6}$</div>



<div id="title">

# Ultraviolet spectroscopy reveals a hot and luminous companion to the Be star + black hole candidate MWC 656

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.14403-b31b1b.svg)](https://arxiv.org/abs/2601.14403)<mark>Appeared on: 2026-01-22</mark> -  _21 pages, 14 figures, submitted to A&A_

</div>
<div id="authors">

<mark>J. Müller-Horn</mark>, et al. -- incl., <mark>K. El-Badry</mark>

</div>
<div id="abstract">

**Abstract:** The Galactic Be star binary MWC 656 was long considered the only known Be star + black hole (BH) system, making it a critical benchmark for models of massive binary evolution and for the expected X-ray emission of Be+BH binaries. However, recent dynamical measurements cast doubt on the presence of a BH companion. We present new multi-epoch ultraviolet spectroscopy from the Hubble Space Telescope (HST), combined with high-resolution optical spectra, to reassess the nature of the companion. The far-ultraviolet spectra reveal high-ionisation features -- including prominent N v and He ii lines -- which are absent in the spectra of normal Be stars and are indicative of a hot, luminous companion. Spectral modelling shows that these features cannot originate from the Be star or from an accretion disc around a compact object. Instead, we find that the data are best explained by a hot ( $T_\mathrm{eff}\approx85 $ kK), compact, hydrogen-deficient star with strong wind signatures, consistent with an intermediate-mass stripped star. Our revised orbital solution and composite spectroscopic modelling yield a companion mass of $M_2 = 1.54^{+0.57}_{-0.46} \mathrm{M}_\odot$ , definitively ruling out a BH and disfavouring a white dwarf. MWC 656 thus joins the growing class of Be + stripped star binaries. The system's unusual properties -- including a high companion temperature and wind strength -- extend the known parameter space of such binaries. The continued absence of confirmed OBe+BH binaries in the Galaxy highlights a growing tension with population synthesis models.

</div>

<div id="div_fig1">

<img src="tmp_2601.14403/./figures/HST_STIS_spec_zoom.png" alt="Fig8" width="100%"/>

**Figure 8. -** HST/STIS UV spectra of MWC 656 obtained on four observing dates. A PoWR stellar atmosphere model with $T_* = 21$ kK, $\log g_* = 3.4$, and $v\sin i = 300$ km s$^{-1}$ is shown in black for comparison. The top and bottom panels show the full spectral range, split into blue and red halves, while the middle row displays zoom-ins on key diagnostic lines: the $\ion${N}{v $\lambda\lambda$1238, 1241} doublet, the $\ion${C}{iv $\lambda\lambda$1548, 1551} doublet, and the $\ion${He}{ii $\lambda$1640} line. Shaded gray regions mark known ISM absorption features. The zoom-in panels reveal prominent high-ionisation features, not present in the Be star model, suggesting they originate from a hotter source within the system. (*fig:spec_uv_zoom*)

</div>
<div id="div_fig2">

<img src="tmp_2601.14403/./figures/HST_STIS_comparison_Be_subdwarfs.png" alt="Fig11.1" width="50%"/><img src="tmp_2601.14403/./figures/HST_STIS_comparison_CVs.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Comparison of the HST/STIS UV spectrum of MWC 656 (black) with reference spectra from two classes of stars: known Be+subdwarf systems \citep[top;][]{Wang+2021}
    and cataclysmic variables \citep[bottom;][]{Pala+2022,Toloza+2023}. The Be+subdwarf and cataclysmic variable spectra have been rebinned using bin sizes of three and two, respectively, to improve S/N. All spectra have been normalised by their median flux, and vertical offsets have been applied for readability. (*fig:stripped_star_comparison_spectra*)

</div>
<div id="div_fig3">

<img src="tmp_2601.14403/./figures/orbit_fit_phasefolded_combined.png" alt="Fig1" width="100%"/>

**Figure 1. -** Phase-folded RV curves of the Be star and its companion in the MWC 656 binary system, using the best-fit orbital period. RV measurements and uncertainties of the Be star (circles) and the companion (squares) are shown with colour-coded markers corresponding to the instrument used.
    Maximum-likelihood orbital models derived from RV fitting are overplotted for the Be star (pink) and the companion (green) with thick lines. Thin lines show orbital models for random samples from the posterior to illustrate uncertainties in the inferred parameters. The grey dotted line shows the inferred systemic velocity. Star symbols indicate the predicted orbital phases and velocities of the four HST/STIS FUV observations based on the fitted orbit. (*fig:orbit_fit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.14403"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# The circumbinary disk of HD 34700A: I. CO gas kinematics indicate spirals, \ infall, and vortex motions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.15262-b31b1b.svg)](https://arxiv.org/abs/2601.15262)<mark>Appeared on: 2026-01-22</mark> -  _22 pages, 20 figures, accepted for publication in Astronomy and Astrophysics on January 16, 2026_

</div>
<div id="authors">

J. Stadler, et al. -- incl., <mark>M. Benisty</mark>, <mark>F. Zagaria</mark>, <mark>D. Fasano</mark>, <mark>N. Kurtovic</mark>

</div>
<div id="abstract">

**Abstract:**            We present the first high-resolution ($\sim$ 0.14") Atacama Large Millimeter/submillimeter Array (ALMA) Band 6 dust continuum and CO molecular line emission observations of the quadruple system HD 34700. In particular, HD 34700AaAb is a spectroscopic binary ($M_{\rm{bin}}=4\,M_\odot$) surrounded by two low-mass companions at large separations. Its circumbinary disk is highly substructured, featuring numerous spiral arms and a large cavity observed in infrared (IR) scattered light. We analyzed the CO line channel and intensity moment maps. By fitting a Keplerian model to the line channel emission, we identified the residual motions and conducted a line spectra analysis. We resolved an asymmetric continuum crescent on top of a dust ring at 0.39" (138 au), colocated with the IR ring. The CO molecule line emissions trace a smaller cavity in gas, whose edge aligns with the inner rim of the ring detected in H$\alpha$ emission at 0.20" (65 au). The $^{12}$CO line emission and kinematics trace highly non-Keplerian motions ($\sim0.1\Delta\upsilon_{\rm kep}$), and these CO spiral features align well with the spiral structures in scattered light. The $^{12}$CO line spectra analysis reveals a streamer above the southeastern disk plane, likely falling onto the disk. The $^{13}$CO and C$^{18}$O kinematics largely follow the disk's underlying Keplerian rotation, while $^{13}$CO exhibits tentative signs of anticyclonic vortex flows at the continuum crescent location. Our multimolecular line study suggests that the circumbinary disk of HD 34700A is highly perturbed in its upper layers, possibly warped and influenced by infalling material. While late-stage infall may account for the IR spirals and the formation of the vortex through Rossby wave instability, an embedded massive companion within the cavity may also contribute to these features.         

</div>

<div id="div_fig1">

<img src="tmp_2601.15262/./figures/HD34700A_overview_allmom_csub_Tbr.png" alt="Fig5" width="100%"/>

**Figure 5. -** Multi-wavelengths observations of \sys . **(a)** SPHERE/IRDIS IR scattered light observations and **(b)** ZIMPOL H$\alpha$ observations presented in \citet{Columba_ea_2024}, with the gray circles representing the coronograph ($r=92.5 $mas $\approx32 $au). **(c)** ALMA 225.3 GHz continuum observations. **(d)**, **(e),** and **(f)** show the moment 0 maps for the \twCO, \thCO,  and \eiCO fiducial cubes, while **(g)**, **(h),** and **(i)** show the peak
intensity maps for the same cubes, respectively. The brightness temperature was computed using the Rayleigh-Jeans approximation. The white contour in the peak intensity maps encloses 5$\sigma$ of the root-mean-square (RMS) noise of each line. The location of the spectroscopic binary is highlighted with the star. The beam size is shown in the lower left corner for the ALMA observations. (*fig:overview*)

</div>
<div id="div_fig2">

<img src="tmp_2601.15262/./figures/CO_centroids_residuals_fix13co_new.png" alt="Fig7" width="100%"/>

**Figure 7. -** Gas kinematics for the CO molecular lines. Centroid LoS Gaussian moment maps for the data (left columns), Keplerian model (center columns), and data-model residuals (right columns), for the \twCO(top) and \thCO(middle) data, using high-resolution cubes. The geometry of \twCO  and \thCO  models was fixed to the \thCO  disk, which is less perturbed and has a higher S/N than \eiCO . For \eiCO(bottom), we employ the fiducial cube. The respective beam sizes are shown in the lower left corners. In \twCO -residuals, we overplotted the IR contours from Fig. \ref{fig:overview}(a). All panels share the same scale, color bars, and are masked at 4$\sigma$ of their peak intensity map. The innermost beam size is masked in as it was excluded from the fit. (*fig:CO_residuals*)

</div>
<div id="div_fig3">

<img src="tmp_2601.15262/./figures/HD34700_line_analysis_chi2.png" alt="Fig6" width="100%"/>

**Figure 6. -** Line spectra diagnostics. Selected \twCO(black) and \thCO(blue) spectra normalized to their peak extracted at disk locations marked in the center map. The map shows the reduced-$\chi^2$ values (Eq. \ref{eq:chi2}) from the Gaussian moment map fit to the line profile (see accurate fit in panel 1), masked at $5\sigma$. High uncertainties indicate non-Gaussian-shaped line profiles. The line profiles are split by the $v_\mathrm{LSRK}$ into regions that are either blue- or redshifted to the systemic velocity. Pixels 6-9 that exhibit secondary components in their spectra that are tracing an "exodisk" line component (cyan vertical lines), in addition to the underlying rotating disk component (purple lines). The black contours show the IR scattered light as in Fig. \ref{fig:overview} (a) and the dotted lines highlight the disk's minor and major axes.  (*fig:line_profiles*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.15262"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A radially broad collisional cascade in the debris disk of $\gamma$ Ophiuchi observed by JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.15285-b31b1b.svg)](https://arxiv.org/abs/2601.15285)<mark>Appeared on: 2026-01-22</mark> -  _20 pages, 9 figures, 2 tables. Submitted to ApJ_

</div>
<div id="authors">

Y. Han, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** The A1V star $\gamma$ Oph, at a distance of 29.7 pc, is known from Spitzer imaging to host a debris disk with a large radial extent and from its spectral energy distribution to host inner warm dust. We imaged $\gamma$ Oph with JWST/MIRI at 15 and 25.5 $\mu$ m, which reveal smooth and radially broad emission that extends to a radius of at least 250 au at 25.5 $\mu$ m. In contrast to JWST findings of an inner small-grain component with distinct ringed substructures in Fomalhaut and Vega, the mid-infrared radial profile combined with prior ALMA imaging suggests a radially broad steady-state collisional cascade with the same grain size distribution throughout the disk. This further suggests that the system is populated by a radially broad planetesimal belt from tens of au or less to well over 200 au, rather than a narrow planetesimal belt from which the observed dust is displaced to appear broad. The disk is also found to be asymmetric, which could be modelled by a stellocentric offset corresponding to a small eccentricity of $\sim$ 0.02. Such a disk eccentricity could be induced by a mildly eccentric $<10 M_\mathrm{Jup}$ giant planet outside 10 au, or a more eccentric companion up to stellar mass at a few au, without producing a resolvable radial gap in the disk.

</div>

<div id="div_fig1">

<img src="tmp_2601.15285/./Fig2.png" alt="Fig5" width="100%"/>

**Figure 5. -** Gallery of nonparametric and parametric models fitted to the MIRI F2550W PSF-subtracted (assuming stellar SED flux) image of $\gamma$ Oph. Each group of 3 panels displays the PSF-convolved disk-only model image (i.e., stellar component not included, displayed on a logarithmic colour scale, to be compared with the top-left panel in Fig. \ref{fig:obs}), disk-only deconvolved and deprojected radial surface brightness (S.B.) and optical depth ($\tau$) profiles and the residual image (data $-$ model). "Offset" indicates that the geometric centre of the disk is offset relative to the star in the model, and is included as (two) free parameters in the model (along two spatial axes). The displacement vector of the disk centre relative to the star is indicated with blue arrows. All radial profile panels are plotted with the same vertical scaling, which span from 0 to 1.8 mJy/arcsec$^2$. For the \texttt{rave} radial profile panel (panel a), the azimuthally averaged profile of the PSF-subtracted (assuming the stellar flux of the best-fit grid point in the 4D \texttt{rave} grid) observations and the PSF-convolved best-fit disk model are overplotted, as this is the 1D quantity that \texttt{rave} directly fits to. Where multiple Gaussian components are invoked (panels e and f), the constituent Gaussian components are individually overplotted. Uncertainties are indicated with shaded regions for the main deconvolved profiles, but these are generally too narrow to be visible for the parametric models. The per-pixel RMS noise measured from the background is 0.07 mJy/arcsec$^2$. North is oriented upwards and east to the left.
     (*fig:models25*)

</div>
<div id="div_fig2">

<img src="tmp_2601.15285/./Fig5.png" alt="Fig7" width="100%"/>

**Figure 7. -** The SED of $\gamma$ Oph. The photometric data points were collected from _Hipparcos_ (Hipparcos1997, [H\og, et. al 2000](https://ui.adsabs.harvard.edu/abs/2000A&A...355L..27H)) , the [ and Mermilliod (2006)](https://ui.adsabs.harvard.edu/abs/2006yCat.2168....0M) UBV catalogue, 2MASS  ([Cutri, et. al 2003](https://ui.adsabs.harvard.edu/abs/2003tmc..book.....C)) , _Gaia_ ([Gaia Collaboration, et. al 2018](https://ui.adsabs.harvard.edu/abs/2018A&A...616A...1G)) , AKARI  ([Ishihara, et. al 2010](https://ui.adsabs.harvard.edu/abs/2010A&A...514A...1I)) , WISE  ([Wright, et. al 2010](https://ui.adsabs.harvard.edu/abs/2010AJ....140.1868W)) , _Spitzer_ ([Su, et. al 2006](https://ui.adsabs.harvard.edu/abs/2006ApJ...653..675S), [Chen, et. al 2014](https://ui.adsabs.harvard.edu/abs/2014ApJS..211...25C), [IRSA and SSC 2020](https://ui.adsabs.harvard.edu/abs/2020ipac.data.I433I)) , _Herschel_ ([Pilbratt, et. al 2010](https://ui.adsabs.harvard.edu/abs/2010A&A...518L...1P), [Pawellek, et. al 2014](https://ui.adsabs.harvard.edu/abs/2014ApJ...792...65P), [Mo\'or, et. al 2015](https://ui.adsabs.harvard.edu/abs/2015MNRAS.447..577M)) , JCMT  ([Holland, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.470.3606H)) , APEX  ([Nilsson, et. al 2010](https://ui.adsabs.harvard.edu/abs/2010A&A...518A..40N))  and ALMA  ([Matr\`a, et. al 2025](https://ui.adsabs.harvard.edu/abs/2025A&A...693A.151M), [Marino, et. al 2026](https://doi.org/10.1051/0004-6361/202556489)) . The $F_*$ points represent the fitted central component flux under then double-power-law-with-offset model at 25 $\mu$m and the Gaussian-with-offset model at 15 $\mu$m. The stellar spectrum corresponds to a Phoenix stellar model  ([Husser, et. al 2013](https://ui.adsabs.harvard.edu/abs/2013A&A...553A...6H))  with $T_\mathrm{eff} = 9050$ K and $\log(g) = 3.9$. The SED models of the disk are overplotted, including those derived using \texttt{rave} on the ALMA 0.87 mm and JWST 25 $\mu$m images, both with the same grain composition and size distribution as described in Section \ref{sec:parametric}.  (*fig:sed*)

</div>
<div id="div_fig3">

<img src="tmp_2601.15285/./Fig7.png" alt="Fig2" width="100%"/>

**Figure 2. -** Constraints on the mass (left axis), eccentricity (right axis) and semimajor axis of a hypothetical planet required to explain the stellocentric offset of the disk. Lower bounds in the mass--semimajor axis parameter space are plotted with thick lines and upper bounds with thin lines. Origins of lower bounds include a sufficiently fast secular perturbation timescale such that the disk becomes eccentric within the age of the system ($t_\mathrm{sec} = \mathrm{age}$), and a sufficiently massive planet to overcome the disk's gravity ($M_\mathrm{disk}$). Upper bounds are set by the lack of a radial gap detected in MIRI observations, either due to the gap being too narrow ($\Delta a_\mathrm{gap}$), or due to a gap having not yet been carved by the planet within the age of the system ($t_\mathrm{clear} = \mathrm{age}$). The shaded region simultaneously satisfies all such constraints described.
    The eccentricity required of the planet to excite a forced eccentricity of 0.02 in the disk at 50 au is overplotted on a separate vertical axis with a black dashed line ($e$).  (*fig:planets*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.15285"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

112  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
